<a href="https://colab.research.google.com/github/Smartsie/Optimizer/blob/Addscaler/Optimizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
""" 
Optim package - T Martin 10 May 2020
This code if for multivariate optimizer using surrogate response surface created by several ML models
The next target point to study is either based on minimum point found, either by using a surrogate if r2 value of 
a model is above a certain threshold
"""

' \nOptim package - T Martin 10 May 2020\nThis code if for multivariate optimizer using surrogate response surface created by several ML models\nThe next target point to study is either based on minimum point found, either by using a surrogate if r2 value of \na model is above a certain threshold\n'

In [4]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, 
and then re-execute this cell.


In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go

In [ ]:
# If we want to export in html in a separate browser
#mport plotly.io as pio
#pio.renderers.default = "browser" 

In [77]:
# Define function to minimize
# x is a vector containing what we want to evaluate
def funct(x):
    x1=x[0]
    x2=x[1]
    #f=x1*x1+x1*x2+2*np.sin((x1+x2)*2)+4*np.cos(x2*2)
    #f=x1*x1+x2*x2-x1*x2+20*np.sin(x1-x2)-8*np.cos(x2*2)
    #f=(x1-3)*(x1-3)+(x2-1)*(x2-1)+x1*x2
    f=(x1-3)*(x2-3)+(x2-1)*(x2-1)+x1*x2+x1-x2
    #f=x[0]**2+x[1]**2-x[2]
    f=x1**4+x2**4-4*x1*x2
    #launch exothermia
    #err1=....
    #err2-....
    #f=err1^2+0.5*err2^2

    return f

In [78]:
# Here we define the range for the xi we want to find. It is the amount of variables
lower_limit=[-5,-5]
upper_limit=[5,5]
boundaries_=[lower_limit, upper_limit]

In [79]:
def Optimtm(eval_function, boundaries, plot_steps=False):
# Import necessary libraries
  from sklearn.gaussian_process import GaussianProcessRegressor
  from sklearn.neural_network import MLPRegressor
  from sklearn.model_selection import train_test_split
  from sklearn.linear_model import LinearRegression
  from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
  from sklearn.preprocessing import MinMaxScaler
  from sklearn.neighbors import KNeighborsRegressor
  from xgboost import XGBRegressor
  from sklearn.model_selection import cross_val_score
  import lightgbm
  import plotly.express as px

  def add_matrix(X,points):
      # This function adds the points to the matrix
      # The matrix is a dataframe
      df=points
      for i in range(len(df)):
          variables=df.iloc[i]
          variables['z']=funct(variables)
          X=X.append(variables)
      return X
  # Define function providing N randoms points in the restrained domain 
  def next_point(current_space,n=5):
      next_space=current_space
      pt=pd.DataFrame()
      for i in range(n):
          vecteur=current_space['Center']
          Radius=current_space['Radius']
          new_vec=vecteur+Radius*np.random.choice((-1,1),size=len(Radius))*(np.random.random(len(Radius)))
          for j in range(len(new_vec)):
              if new_vec[j]<space.iloc[j]['Center']-space.iloc[j]['Radius']:
                  new_vec[j]=space.iloc[j]['Center']-space.iloc[j]['Radius']
              if new_vec[j]>space.iloc[j]['Center']+space.iloc[j]['Radius']:
                  new_vec[j]=space.iloc[j]['Center']+space.iloc[j]['Radius']                              
          pt['n'+str(i)]=new_vec
      pt=pt.T
      pt.columns=X.drop('z',axis=1).columns
      #next_space['Radius']=next_space['Radius']*epsilon   <------ A vérifier!!
      return pt
  def training_loop(X,domain,AI=False,n=5,epsilon=0.95):
    # Training loop: 
    # 1-Select next target points to evaluate in the live domain and evaluate them
      df=next_point(domain,n)
      Matrix=add_matrix(X,df)
      x=Matrix.drop('z',axis=1).values
      y=Matrix['z'].values
    # 2-if AI = True, then we evaluate a set of ML models
      if AI==True:
          # Need to do a CV split with Train test split
          models=[LinearRegression(), KNeighborsRegressor (n_neighbors=5),lightgbm.LGBMRegressor(),GaussianProcessRegressor(random_state=0),XGBRegressor(objective='reg:squarederror')]
          # Additional models: MLPRegressor(hidden_layer_sizes=(5,3),max_iter=300, verbose=False,learning_rate_init=0.05),
          scalerx=MinMaxScaler()
          scalery=MinMaxScaler()
          x_scaled=scalerx.fit_transform(x)
          y_scaled=scalery.fit_transform(y.reshape(-1, 1))
          r2max=-100
          best_model=models[1]
          for model_loop in models:
            score=cross_val_score(model_loop,x_scaled,np.ravel(y_scaled),cv=4)
            r2=np.mean(score)     
            if r2>r2max:
              r2max=r2
              best_model=model_loop 
          if r2max<0.7:
            print('R2 insufficient %0.2f'%r2max,', proceeding without AI')
            theor_min=np.argmin(X.z.values)
            lowest_value=np.min(X.z.values)
            minimum=X.iloc[theor_min]
          else:
              model=best_model
              print(type(model).__name__, " has been found with r2=%0.2f"%r2max)
              # Retrain the model with all data
              model.fit(x_scaled,y_scaled)
              # Predict and find the minimum in one shot
              Prediction=model.predict(scalerx.transform(srf_x.values))
              lowest_value_ai_predicted=np.min(Prediction)
              minimum=srf.iloc[np.argmin(Prediction)]    
              # Evaluate at minimum
              lowest_value=funct(minimum.values)  
      else:
          theor_min=np.argmin(X.z.values)
          lowest_value=np.min(X.z.values)
          minimum=X.iloc[theor_min]
      minimum=minimum.drop('z')
      # New space should be centered around this minimum
      domain['Center']=(minimum.values*0.5+domain.Center*0.5)
      domain['Radius']=domain['Radius']*epsilon
      return Matrix,domain,lowest_value

  nvar=len(boundaries[0]) #How many variables we want to optimize
  # Initialize the space domain. Instead of [-5,5] we convert to Center + Radius [0,5]
  space=pd.DataFrame(columns=['Center','Radius'])
  for i in np.arange(0,nvar):
    space.loc[i]=[(boundaries[0][i]+boundaries[1][i])/2,0.5*(boundaries[1][i]-boundaries[0][i])]
  # Initialize domain
  domain=space.copy()

  # Initialize X matrix. It will contain all evaluated points. We start with 5 evaluations points:
  X=pd.DataFrame()
  for i in np.arange(0,nvar):
      X['x'+str(i)]=0
     # Fill X with 5 points picked at domain boundary
  for i in np.arange(0,nvar):
    X['x'+str(i)]=[boundaries[0][i],boundaries[0][i],boundaries[1][i],boundaries[1][i],(boundaries[0][i]+boundaries[1][i])/2,(boundaries[0][i]+boundaries[1][i])/2,(boundaries[0][i]+boundaries[1][i])/2,boundaries[0][i],boundaries[1][i]]
  X['z']=0
     # Evaluate the points  
  Z=[]
  for i in np.arange(0,len(X)):
      variables=X.drop(['z'],axis=1).iloc[i]
      #variables['target']=funct(variables[['x0','x1']].values)
      z=eval_function(variables.values)
      Z.append(z)
  X['z']=Z

  # Initialize Evaluation surface. We generate N points for evaluating the ML model. N can be big (1000)
  N=10000
  srf=pd.DataFrame()
  loc=0
  # We will sample 100 points from the boundary. This 100 gives the granularity.
  for i in np.arange(0,nvar):
    xi=np.random.choice(np.linspace(boundaries[0][i],boundaries[1][i],num=100),N)
    srf['x'+str(i)]=xi
  Z=[]
  for i in np.arange(0,len(srf)):
    z=funct(srf.iloc[i].values)
    Z.append(z)
  srf['z']=Z
  # for japan srf=DF
  srf_x=srf.drop(['z'],axis=1)
  # Plot evaluation surface
  plot_srf=True
  if plot_srf==True:
    fig = px.scatter_3d(srf, x='x0', y='x1', z='z',color='z')
    #fig.show()
  print("Minimum found on evaluation surface : ")
  index=np.argmin(srf.z)
  print(srf.iloc[index])
 
  # Run the training loop
  N_loops=20
  Max_points=20
  point_amount=Max_points
  Dom=[] #pd.DataFrame(columns=['x0 C','x0 R','x1 C','x1 R'])]
  for i in np.arange(0,N_loops):
      # Point amount should decrease. At the beginning, it should be 10. At the end, 2.
      point_amount=np.int((Max_points-3)-(Max_points-3)*i/N)+3

      X,domain,minimum=training_loop(X,domain,AI=True,n=point_amount,epsilon=0.8)
      minimum=np.min(X.z.values)
      #print("epoch ",i," Minimum=%0.2f"%minimum)
  min_found=np.min(X.z.values)
  best_point=X.iloc[np.argmin(X.z.values)]
  if plot_srf==True:
    fig.update_traces(mode='markers', marker_line_width=2, marker_size=5)
    fig2=px.scatter_3d(X,x='x0',y='x1',z='z',color_discrete_sequence=['red'])
    fig2.update_traces(mode='markers', marker_line_width=2, marker_size=8)
    fig3=px.scatter_3d(pd.DataFrame(best_point).T,x='x0',y='x1',z='z',color_discrete_sequence=['green'])
    fig3.update_traces(mode='markers', marker_line_width=2, marker_size=15)
    fig.add_trace(fig2.data[0])
    fig.add_trace(fig3.data[0])
    fig.show()
  return X,min_found, best_point

In [80]:
X,minimum, best_point=Optimtm(funct,boundaries_,True)
print(best_point)

Minimum found on evaluation surface : 
x0   -0.959596
x1   -0.959596
z    -1.987462
Name: 5263, dtype: float64
GaussianProcessRegressor  has been found with r2=0.84
GaussianProcessRegressor  has been found with r2=0.99
GaussianProcessRegressor  has been found with r2=1.00
GaussianProcessRegressor  has been found with r2=0.99
GaussianProcessRegressor  has been found with r2=0.99
GaussianProcessRegressor  has been found with r2=0.99
GaussianProcessRegressor  has been found with r2=0.99
GaussianProcessRegressor  has been found with r2=0.99
GaussianProcessRegressor  has been found with r2=0.99
GaussianProcessRegressor  has been found with r2=0.99
GaussianProcessRegressor  has been found with r2=0.99
GaussianProcessRegressor  has been found with r2=0.99
GaussianProcessRegressor  has been found with r2=0.99
GaussianProcessRegressor  has been found with r2=0.99
GaussianProcessRegressor  has been found with r2=0.99
GaussianProcessRegressor  has been found with r2=0.99
GaussianProcessRegressor 

x0   -1.001679
x1   -1.007605
z    -1.999685
Name: n1, dtype: float64


In [81]:
pd.DataFrame(best_point).T

,x0,x1,z
n1,-1.001679,-1.007605,-1.999685


In [60]:
# Print Minimum found
min=np.min(X.z.values)
print("Minimum found:")
print(X.iloc[np.argmin(X.z.values)])
#print("Theoretical Minimum : ")
#print(srf.iloc[np.argmin(srf.z)])


Minimum found:
x0    -5.0
x1     5.0
z    -35.0
Name: n15, dtype: float64


In [61]:
np.ravel(y)

array([1, 2])

In [62]:
y=[[1],[2]]

In [63]:
y.ravel()

AttributeError: ignored

In [ ]:
y.ravel